# Plotting with PyQt

In [1]:
import numpy as np
import nolds
import scipy.io
import mne
import pandas as pd
import seaborn as sns
import pyqtgraph as pg
from scipy.io import loadmat
from scipy import stats
import matplotlib.pyplot as plt
import antropy as ent
from attractors import Attractor
from antropy import higuchi_fd
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.computation import RQAComputation
from pyrqa.time_series import TimeSeries
from pyrqa.result import RQAResult
from pyrqa.opencl import OpenCL
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.neighbourhood import FixedRadius
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.image_generator import ImageGenerator
import numpy as np
import matplotlib.pyplot as plt
from antropy import higuchi_fd

In [6]:
# Load data
data = loadmat('/home/vincent/AAA_projects/MVCS/Neuroscience/downsampled/EEG_DS_Struct_0101.mat')
stim_data = pd.read_excel('/home/vincent/AAA_projects/MVCS/Neuroscience/EEG-tES-Chaos-Neural-Net/Stimulation.xlsx')

# Fill null values in 'Sub#' column
stim_data['Sub#'].fillna(method='ffill', inplace=True)

# Drop the first row
stim_data = stim_data.drop(0)

DSamp = data['DSamp']

# Get data parameters
triggers = DSamp[0][0][0]
EEGdata = DSamp[0][0][1]
fs = DSamp[0][0][2][0][0]
fsOld = DSamp[0][0][3][0][0]
time = DSamp[0][0][4][0]
label = DSamp[0][0][5]
nchan = DSamp[0][0][6][0][0]
rate = DSamp[0][0][7][0][0]
npt = DSamp[0][0][8][0][0]
Subj = DSamp[0][0][9][0]
ptrackerPerf = DSamp[0][0][10]
ptrackerTime = DSamp[0][0][11]
ptrackerfs = DSamp[0][0][12][0][0]

# List of unwanted channel names
unwanted_channels = ['BIP1', 'BIP2', 'RESP1']

# Filter out unwanted channels from the label data
filtered_label = [ch for ch in label if ch[0][0] not in unwanted_channels]

# Convert the filtered list back to numpy array and replace the original label
label = np.array(filtered_label, dtype=object)

print(triggers, EEGdata, label, stim_data)

[[(array([[20.428]]), array([[20429]], dtype=uint16), array(['0002'], dtype='<U4'), array([[2]], dtype=uint8), array(['Block Start'], dtype='<U11'), array([], shape=(1, 0), dtype=float64))
  (array([[619.442]]), array([[619443]], dtype=int32), array(['0002'], dtype='<U4'), array([[2]], dtype=uint8), array(['Block Start'], dtype='<U11'), array([], shape=(1, 0), dtype=float64))
  (array([[619.499]]), array([[619500]], dtype=int32), array(['0016'], dtype='<U4'), array([[16]], dtype=uint8), array(['Stim Start'], dtype='<U10'), array(['M30'], dtype='<U3'))
  (array([[654.746]]), array([[654747]], dtype=int32), array(['0032'], dtype='<U4'), array([[32]], dtype=uint8), array(['Stim Stop'], dtype='<U9'), array([], shape=(1, 0), dtype=float64))
  (array([[770.515]]), array([[770516]], dtype=int32), array(['0016'], dtype='<U4'), array([[16]], dtype=uint8), array(['Stim Start'], dtype='<U10'), array(['M30'], dtype='<U3'))
  (array([[805.571]]), array([[805572]], dtype=int32), array(['0032'], dtyp

In [ ]:
import pyqtgraph as pg
from PyQt5.QtWidgets import QApplication
from pyqtgraph.Qt import QtCore
import numpy as np
import sys

# Number of channels
n = 8

# Initialize the QApplication
app = QApplication([])

# Create the window
win = pg.GraphicsWindow()
win.setWindowTitle('EEG Scrolling Plot')

# Create a PlotItem in the GraphicsWindow
p = win.addPlot()

# Create an empty list of PlotDataItems
curves = [p.plot(pen=(i,n)) for i in range(n)]

# Start with no data
data = np.empty((n,0))

def update():
    global data, curves

    # Add a new column of random data
    new_data = np.random.randn(n,1) / 10
    data = np.hstack((data, new_data))

    # Update the PlotDataItems with the new data
    for i in range(n):
        curves[i].setData(data[i])

# Set up a timer to update the plot every 100ms
timer = QtCore.QTimer()
timer.timeout.connect(update)
timer.start(100)

# Start the QApplication
if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
    QApplication.instance().exec_()

In [ ]:
print(data.keys())

In [ ]:
subjects_data = data['DSamp']  # This now represents multiple subjects

# Initialize an empty list to store the mean fractal dimension for each subject
fractal_dims_all_subjects = []
subjects_group = []  # Add this line to track the group assignments

# Loop over each subject's data
for subject_idx in range(subjects_data.shape[0]):
    print(f"Processing Subject {subject_idx}")  # Printout for tracking progress
    
    # Extract EEG data for the current subject
    EEG_data = subjects_data[subject_idx][0][1]

    # Reshape EEG data to 1D if necessary
    eeg_data = EEG_data.flatten()

    # Split the data into segments of equal length
    segment_length = int(0.10 * len(eeg_data))
    segments = [eeg_data[i:i+segment_length] for i in range(0, len(eeg_data), segment_length)]
    
    # Check if no segments exist or any segment is too short
    if not segments or len(segments[0]) < 2: 
        print(f"Subject {subject_idx} has insufficient data.")
        continue
    
    # If no segments exist or any segment is too short, continue to the next subject
    if not segments or len(segments[0]) < 2: 
        print(f"Subject {subject_idx} has insufficient data.")
        continue
    
    # Calculate the fractal dimension of each segment
    try:
        fractal_dims = [nolds.dfa(segment) for segment in segments if len(segment) >= 2]
    except ValueError:
        print(f"Subject {subject_idx} cannot calculate DFA.")
        continue

    # Calculate the mean fractal dimension across all segments
    mean_fractal_dim = np.mean(fractal_dims)
    
    # Store the mean fractal dimension in the list
    fractal_dims_all_subjects.append(mean_fractal_dim)
    
    # Use the mean fractal dimension to stratify the subject
    if mean_fractal_dim < 1.2:  
        print(f"Subject {subject_idx} belongs to group A")
        subjects_group.append('A')  # Add this line to track the group assignments
    else:
        print(f"Subject {subject_idx} belongs to group B")
        subjects_group.append('B')  # And this line too

# Convert to a numpy array for convenience
fractal_dims_all_subjects = np.array(fractal_dims_all_subjects)
subjects_group = np.array(subjects_group)  # Convert this to a numpy array too

# Print some basic statistics
print(f"Mean fractal dimension across all subjects: {np.mean(fractal_dims_all_subjects)}")
print(f"Standard deviation of fractal dimension across all subjects: {np.std(fractal_dims_all_subjects)}")

# Histogram of the fractal dimensions
sns.histplot(fractal_dims_all_subjects, bins=10, kde=True)
plt.xlabel('Fractal Dimension')
plt.ylabel('Count')
plt.title('Histogram of Fractal Dimensions across all subjects')
plt.show()

# Perform a statistical test if applicable (e.g., t-test, ANOVA)
# This is just a placeholder. Adjust according to your research question and data.
# For example, if you have two groups (A and B), you could do a t-test.
group_A_fractal_dims = fractal_dims_all_subjects[subjects_group == 'A']
group_B_fractal_dims = fractal_dims_all_subjects[subjects_group == 'B']
t_stat, p_val = stats.ttest_ind(group_A_fractal_dims, group_B_fractal_dims)
print(f"t-statistic: {t_stat}, p-value: {p_val}")